In [45]:
import os
import json
import pandas as pd
import traceback



In [46]:
from langchain_openai import ChatOpenAI

In [47]:
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
KEY=os.getenv("api_key")

In [49]:
llm=ChatOpenAI(
               model="deepseek/deepseek-r1:free",   
               openai_api_key=KEY ,  
               openai_api_base="https://openrouter.ai/api/v1",
               temperature=0.7,          
                )

In [50]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001DEF24A2490>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001DEF284CB90>, root_client=<openai.OpenAI object at 0x000001DEF24A3B10>, root_async_client=<openai.AsyncOpenAI object at 0x000001DEF24A2350>, model_name='deepseek/deepseek-r1:free', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

In [51]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain  
import PyPDF3 

In [52]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [53]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.

### RESPONSE_JSON
{response_json}
 "IMPORTANT: Respond with only valid JSON exactly matching RESPONSE_JSON. Do NOT add any explanation."

"""

In [54]:

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [55]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [56]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [57]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [58]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)

In [59]:

from langchain.schema.runnable import RunnableSequence   
generate_evaluate_chain = quiz_chain | review_chain

#generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
  #                                      output_variables=["quiz", "review"], verbose=True,)

In [60]:
file_path =r"C:\Users\Hp\mcqgen\data.txt"

In [61]:
file_path  

'C:\\Users\\Hp\\mcqgen\\data.txt'

In [62]:
with open(file_path, "r", encoding="utf-8") as file:
    TEXT = file.read()

In [63]:
print(TEXT)

Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.[7][14][15]

Generative AI has

In [64]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [65]:
NUMBER=5
SUBJECT="generativeai"
TONE="simple"

In [66]:
inputs = {
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON),
}
response = generate_evaluate_chain.invoke(inputs)  
response 



> Entering new LLMChain chain...
Prompt after formatting:

Text: Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Mi

AuthenticationError: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}

In [ ]:
import json

raw_quiz = response.get("quiz", "").strip()

# Try parsing JSON safely
try:
    if raw_quiz.startswith("```"):
        raw_quiz = raw_quiz.strip("`")
        if raw_quiz.startswith("json"):
            raw_quiz = raw_quiz[4:].strip()
    
    quiz = json.loads(raw_quiz)
except json.JSONDecodeError as e:
    print("Failed to decode quiz JSON:", e)
    print("Raw output was:", raw_quiz)
    quiz = {}


In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [ ]:
quiz_table_data

[{'MCQ': 'What is the primary function of generative AI models?',
  'Choices': 'a: Analyze existing data patterns | b: Generate new data based on input | c: Replace human jobs entirely | d: Control robotic systems',
  'Correct': 'b'},
 {'MCQ': 'Which technology was the first example of algorithmically generated media?',
  'Choices': 'a: Neural networks | b: Markov chains | c: Transformers | d: Deep learning',
  'Correct': 'b'},
 {'MCQ': 'Which of these is a text-to-image generative AI model?',
  'Choices': 'a: ChatGPT | b: DeepSeek | c: Stable Diffusion | d: Sora',
  'Correct': 'c'},
 {'MCQ': 'What is a major ethical concern associated with generative AI?',
  'Choices': 'a: Enhancing customer service | b: Mass replacement of human jobs | c: Improving healthcare diagnosis | d: Generating educational content',
  'Correct': 'b'},
 {'MCQ': 'Which method was used in 1980s-90s generative AI planning systems?',
  'Choices': 'a: Neural networks | b: State space search | c: Deep learning | d: T

In [ ]:
from datetime import datetime
f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

'08_21_2025_21_38_51.log'